In [341]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import GIN_model
importlib.reload(GIN_model)
from GIN_model import ShallowGNNNet
from torchinfo import summary

In [342]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowGNNNet(in_chans,n_classes,input_window_samples)

In [343]:
print(model)

ShallowGNNNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (gnn): GNN(
    (gcnConv): GCNConv(10, 10)
  )
  (batch_norm): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 100), stride=(1, 100), padding=0)
  (dropout): Dropout(p=0.8, inplace=False)
  (fc): Linear(in_features=2420, out_features=4, bias=True)
)


In [344]:
for name in model.parameters():
    print(f"Parameter name: {name.shape}")

for name, param in model.named_parameters():
    print(name)


Parameter name: torch.Size([10, 1, 1, 25])
Parameter name: torch.Size([10])
Parameter name: torch.Size([484])
Parameter name: torch.Size([10])
Parameter name: torch.Size([10, 10])
Parameter name: torch.Size([22])
Parameter name: torch.Size([22])
Parameter name: torch.Size([4, 2420])
Parameter name: torch.Size([4])
temporal.weight
temporal.bias
gnn.edge_weights
gnn.gcnConv.bias
gnn.gcnConv.lin.weight
batch_norm.weight
batch_norm.bias
fc.weight
fc.bias


In [345]:
summary(model,input_size=(2, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Parameter containing:
tensor([1.4708, 1.8743, 1.9763, 0.9695, 1.6004, 0.9428, 1.8342, 0.6575, 0.2181,
        0.8955, 1.5320, 1.7870, 0.2157, 1.8926, 1.9036, 1.7677, 0.5124, 0.4187,
        1.8753, 0.0886, 1.1408, 0.2079, 1.7410, 1.9145, 1.1016, 0.6151, 1.0655,
        0.1817, 0.3617, 1.5850, 1.0945, 0.5254, 1.2622, 1.9390, 0.6630, 0.4392,
        1.2819, 0.1864, 1.7497, 0.7539, 1.3475, 1.2986, 1.8432, 1.1850, 1.6393,
        1.2204, 1.4978, 1.9106, 1.8553, 0.8302, 0.4429, 0.1149, 0.7730, 0.8092,
        0.7541, 1.2996, 0.4612, 0.7304, 0.4511, 0.3225, 1.7240, 1.6518, 1.5924,
        1.8324, 0.8589, 1.8901, 1.1169, 0.9272, 1.1478, 0.7652, 1.9080, 0.7196,
        0.1520, 0.5508, 1.0226, 0.1045, 1.3828, 1.8010, 1.0639, 0.4566, 0.4554,
        1.5341, 1.1277, 1.8871, 0.0449, 1.1576, 0.2888, 0.0838, 0.9101, 0.5478,
        1.9929, 1.1884, 1.3739, 1.5401, 1.8642, 0.7589, 0.9543, 1.5603, 1.4824,
        1.8056, 0.9584, 1.0116, 0.0343, 1.7953, 0.3093, 1.1209, 0.2875, 1.1032,
        0.5388, 1.

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowGNNNet                            [2, 22, 1125]             [2, 4]                    --                        --
├─Conv2d: 1-1                            [2, 1, 22, 1125]          [2, 10, 22, 1101]         260                       [1, 25]
├─GNN: 1-2                               [48444, 10]               [48444, 10]               484                       --
│    └─GCNConv: 2-1                      [48444, 10]               [48444, 10]               10                        --
│    │    └─Linear: 3-1                  [48444, 10]               [48444, 10]               100                       --
│    │    └─SumAggregation: 3-2          [48906, 10]               [48444, 10]               --                        --
├─BatchNorm2d: 1-3                       [2, 22, 10, 1101]         [2, 22, 10, 1101]         44                        --
├─AvgPool